In [1]:
import sys
sys.path.append('/home/cryptobrahman/Own/football_competitions_research/own_modules/') 

In [2]:
import sys
sys.path.append('/mnt/KINGSTON_120/Own/football_competitions_research/own_modules/') 

In [3]:
import pandas as pd
import numpy as np

from flatlib import const
from flatlib import aspects
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.object import GenericObject
from flatlib.tools.chartdynamics import ChartDynamics

from flatlib.tools import arabicparts
from flatlib.dignities import essential

from _astro_constants import AstrologicalConstants, AstrologicalPoints, TransformDoubleValues

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
df = pd.DataFrame(data)
df

,date,utc_time,lon,lat
0,2015/03/13,14:00,38n32,8w54
1,2016/05/23,12:30,08n07,18e14
2,2018/10/03,00:15,14s42,36w04


In [6]:
df['charts'] = AstrologicalPoints.charts_calculate(df, 'date', 'utc_time', 'lon', 'lat')

In [7]:
df.charts

0    <flatlib.chart.Chart object at 0x7f3c869f1490>
1    <flatlib.chart.Chart object at 0x7f3c869f14c0>
2    <flatlib.chart.Chart object at 0x7f3c869ff310>
Name: charts, dtype: object

In [8]:
necessary_constants = AstrologicalConstants.necessary_constants
str(necessary_constants)

"['SUN', 'MOON', 'SATURN', 'URANUS', 'NEPTUNE', 'PLUTO', 'CHIRON', 'NORTH_NODE', 'SOUTH_NODE']"

In [9]:
rulers_constants = AstrologicalConstants.rulers_constants
str(rulers_constants)

"['ASC', 'DESC', 'MC', 'IC', 'PARS_FORTUNA']"

In [10]:
antes_objects = AstrologicalConstants.for_antes_objects
str(antes_objects)

"['moon', 'ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'pars_fortuna', 'uranus', 'neptune', 'pluto', 'chiron']"

In [11]:
pars_constants = AstrologicalConstants.pars_constants
str(pars_constants)

"['PARS_SPIRIT', 'PARS_GLORY', 'PARS_CREST', 'PARS_ROCK']"

In [12]:
for obj in rulers_constants + necessary_constants + pars_constants:
    df[str.lower(obj)] = AstrologicalPoints.calculate_astro_objects(df.charts, obj)

In [13]:
pars_constants.append('ANTES_PARS_FORTUNA')

In [21]:
for obj in antes_objects:
    antes_name = 'antes_' + obj
    df[antes_name] = AstrologicalPoints.antes_objects(df, obj)

In [16]:
antes_objects = []

for col in df.columns:
    if col.startswith('antes'):
        AstrologicalPoints.rename_antes_objects(df, col)
        antes_objects.append(col)      

In [17]:
rulers_constant_col_names = AstrologicalPoints.rulers_col_names(rulers_constants)
rulers_constant_col_names

['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna']

In [18]:
rulers_pars_col_names = AstrologicalPoints.rulers_col_names(pars_constants)
rulers_pars_col_names.append('ruler_antes_pars_fortuna')
str(rulers_pars_col_names)

"['ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock', 'ruler_antes_pars_fortuna', 'ruler_antes_pars_fortuna']"

In [19]:
for name, obj in zip(rulers_constant_col_names, rulers_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [23]:
# for name, obj in zip(rulers_pars_col_names, pars_constants):
#     obj = str.lower(obj)
#     df[name] = AstrologicalPoints.ruler_of_object(df[obj])
#     df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [26]:
houses_objects = AstrologicalConstants.houses_objects
houses_objects

['asc', 'desc', 'mc', 'ic']

In [27]:
str(antes_objects)

"['antes_moon', 'antes_pars_fortuna', 'antes_uranus', 'antes_neptune', 'antes_pluto', 'antes_chiron']"

In [28]:
pars_objects = AstrologicalConstants.pars_objects
str(pars_objects)

"['pars_fortuna', 'pars_spirit', 'pars_glory', 'pars_crest', 'pars_rock']"

In [29]:
ruler_objects = AstrologicalConstants.ruler_objects
str(ruler_objects)

"['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna', 'ruler_antes_pars_fortuna', 'ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock']"

In [30]:
main_objects = AstrologicalConstants.main_objects
str(main_objects)

"['moon', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [33]:
cols_for_id = houses_objects + antes_objects + pars_objects + ruler_objects + main_objects 
print(len(cols_for_id))
str(cols_for_id)

34


"['asc', 'desc', 'mc', 'ic', 'antes_moon', 'antes_pars_fortuna', 'antes_uranus', 'antes_neptune', 'antes_pluto', 'antes_chiron', 'pars_fortuna', 'pars_spirit', 'pars_glory', 'pars_crest', 'pars_rock', 'ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna', 'ruler_antes_pars_fortuna', 'ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock', 'moon', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [26]:
df['id_for_aspects'] = AstrologicalPoints.id_for_aspekts(df, cols_for_id)

In [304]:
df['transform_id'] = df.id_for_aspects.map(lambda x: TransformDoubleValues(x).conbine_class_methods())

In [305]:
df.drop(columns='id_for_aspects', inplace=True)

In [25]:
df #34

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,pars_spirit,pars_glory,pars_crest,pars_rock,antes_moon,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,ruler_pars_spirit,ruler_pars_glory,ruler_pars_crest,ruler_pars_rock,ruler_antes_pars_fortuna
0,2015/03/13,14:00,38n32,8w54,<flatlib.chart.Chart object at 0x7f38a812d490>,<Asc Cancer +27:24:38>,<Desc Capricorn +27:24:38>,<MC Aries +13:01:44>,<IC Libra +13:01:44>,<Pars Fortuna Aries +25:28:18>,<Sun Pisces +22:39:56 +00:59:52>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Uranus Aries +15:06:33 +00:03:14>,<Neptune Pisces +07:50:38 +00:02:13>,<Pluto Capricorn +15:14:35 +00:01:02>,<Chiron Pisces +17:36:15 +00:03:43>,<North Node Libra +11:08:52 -00:03:11>,<South Node Aries +11:08:52 -00:03:11>,<Pars Spirit Libra +29:20:59>,<Pars Glory Sagittarius +18:23:55>,<Pars Crest Sagittarius +08:56:02>,<Pars Rock Aries +09:40:27>,<Antes Moon Capricorn +09:16:25 +13:13:02>,<Antes Pars Fortuna Virgo +04:31:42>,<Antes Uranus Virgo +14:53:27 +00:03:14>,<Antes Neptune Libra +22:09:22 +00:02:13>,<Antes Pluto Sagittarius +14:45:25 +00:01:02>,<Antes Chiron Libra +12:23:45 +00:03:43>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Mars Aries +16:27:08 +00:45:19>,<Venus Aries +25:21:06 +01:12:41>,<Mars Aries +16:27:08 +00:45:19>,<Venus Aries +25:21:06 +01:12:41>,<Jupiter Leo +13:39:12 -00:04:47>,<Jupiter Leo +13:39:12 -00:04:47>,<Mars Aries +16:27:08 +00:45:19>,<Mercury Pisces +00:37:44 +01:29:39>
1,2016/05/23,12:30,08n07,18e14,<flatlib.chart.Chart object at 0x7f38a812d4c0>,<Asc Virgo +27:12:45>,<Desc Pisces +27:12:45>,<MC Gemini +27:30:28>,<IC Sagittarius +27:30:28>,<Pars Fortuna Aries +15:36:29>,<Sun Gemini +02:47:58 +00:57:39>,<Moon Sagittarius +21:11:41 +12:19:41>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Uranus Aries +22:47:40 +00:02:49>,<Neptune Pisces +11:54:55 +00:00:42>,<Pluto Capricorn +17:11:35 -00:00:57>,<Chiron Pisces +24:43:00 +00:01:47>,<North Node Virgo +18:00:29 -00:03:11>,<South Node Pisces +18:00:29 -00:03:11>,<Pars Spirit Pisces +08:49:02>,<Pars Glory Capricorn +07:57:50>,<Pars Crest Virgo +14:43:46>,<Pars Rock Aries +08:19:08>,<Antes Moon Capricorn +08:48:19 +12:19:41>,<Antes Pars Fortuna Virgo +14:23:31>,<Antes Uranus Virgo +07:12:20 +00:02:49>,<Antes Neptune Libra +18:05:05 +00:00:42>,<Antes Pluto Sagittarius +12:48:25 -00:00:57>,<Antes Chiron Libra +05:17:00 +00:01:47>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mars Sagittarius +01:25:22 -00:21:00>,<Jupiter Virgo +13:33:03 +00:02:30>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Mercury Taurus +14:22:23 +00:04:22>,<Mars Sagittarius +01:25:22 -00:21:00>,<Mercury Taurus +14:22:23 +00:04:22>
2,2018/10/03,00:15,14s42,36w04,<flatlib.chart.Chart object at 0x7f38a813b310>,<Asc Gemini +05:25:08>,<Desc Sagittarius +05:25:08>,<MC Pisces +07:39:39>,<IC Virgo +07:39:39>,<Pars Fortuna Pisces +13:18:26>,<Sun Libra +09:44:33 +00:59:03>,<Moon Cancer +17:37:52 +14:05:52>,<Saturn Capricorn +03:06:46 +00:02:33>,<Uranus Taurus +01:23:25 -00:02:14>,<Neptune Pisces +14:24:42 -00:01:27>,<Pluto Capricorn +18:45:26 +00:00:03>,<Chiron Pisces +29:41:06 -00:02:40>,<North Node Leo +02:19:59 -00:03:11>,<South Node Aquarius +02:19:59 -00:03:11>,<Pars Spirit Leo +27:31:49>,<Pars Glory Aries +22:42:34>,<Pars Crest Taurus +01:46:57>,<Pars Rock Pisces +12:02:55>,<Antes Moon Gemini +12:22:08 +14:05:52>,<Antes Pars Fortuna Libra +16:41:34>,<Antes Uranus Leo +28:36:35 -00:02:14>,<Antes Neptune Libra +15:35:18 -00:01:27>,<Antes Pluto Sagittarius +11:14:34 +00:00:03>,<Antes Chiron Libra +00:18:54 -00:02:40>,<Mercury Libra +18:46:13 +01:38:51>,<Jupiter Scorpio +22:27:07 +00:11:21>,<Jupiter Scorpio +22:27:07 +00:11:21>,<Mercury Libra +18:46:13 +01:38:51>

In [307]:
chart = df.charts.values[0]
chart

In [26]:
antes_sun = chart.getObject(const.SUN).antiscia()
# antes_sun.id = 'Antes_Sun'

In [27]:
print(antes_sun)

<Sun Libra +07:20:04 +00:59:52>


In [28]:
dyn = ChartDynamics(chart)

In [29]:
moon_asps = dyn.immediateAspects(const.MOON, const.MAJOR_ASPECTS)
print(moon_asps)  

({'id': 'Mars', 'asp': 120, 'orb': 4.274145169554572}, {'id': 'Sun', 'asp': 90, 'orb': 1.9390151432681364})


In [31]:
moon_asc_aspect = aspects.getAspect(chart.get(const.MOON), chart.get(const.MARS), const.MAJOR_ASPECTS)
print('moon_asc_aspect: ', moon_asc_aspect)

moon_asc_aspect:  <Moon Mars 120 Separative +04:16:27>


In [33]:
moon_aspects = aspects.getAspect(chart.get(const.MOON), antes_sun, const.MAJOR_ASPECTS)
print('moon_aspects: ', moon_aspects)

moon_aspects:  <Moon Sun -1 None +00:00:00>


In [ ]:
# Create new columns for each object